In [1]:
%load_ext blackcellmagic
import pandas as pd
import random

### Generate a set of tasks and the time that each takes

In [2]:
num_tasks = 20

tasks = pd.DataFrame(index=list(range(1, num_tasks+ 1)))
tasks['Time'] = [random.randint(1, 5) for i in range(len(tasks))]
tasks.index.name = 'Task'
tasks

,Time
Task,
1,4
2,3
3,1
4,3
5,3
6,3
7,5
8,1
9,2


### Generate Resource Groups

In [11]:
resource_groups_list = ["Tiger", "Lion", "Cougar"]
resource_groups = pd.DataFrame(index=resource_groups_list).assign(
    MinTasks=[
        random.randint(2, int(num_tasks / (2 * len(resource_groups_list))))
        for i in resource_groups_list
    ],
    MaxTasks=lambda df: [
        random.randint(
            it.MinTasks, max(it.MinTasks, int(2 * num_tasks / len(resource_groups_list)))
        )
        for it in df.itertuples()
    ],
)
resource_groups.index.name = "ResourceGroup"
display(resource_groups)

,MinTasks,MaxTasks
ResourceGroup,,
Tiger,2,12
Lion,2,3
Cougar,3,10


### Generate a set of resources, the number of hours each take, and the cost per hour for each resource

In [6]:
num_resources = 6
resources = pd.DataFrame(index=[chr(65 + i) for i in range(num_resources)])
resources['HoursAvailable'] = [random.randint(6, 15) for i in range(len(resources))]
resources['CostPerHour'] = [random.randint(15, 30) for i in range(len(resources))]
resources['ResourceGroup'] = [random.choice(resource_groups_list) for i in range(len(resources))]
resources.index.name = 'Resource'
resources

,gamma,lambda,S
Resource,,,
A,9,21,Cougar
B,15,22,Tiger
C,13,29,Lion
D,14,28,Tiger
E,13,24,Tiger
F,7,18,Cougar


### Generate the set of tasks that each resource can perform

In [7]:
avg_percent_of_tasks_for_resource = 0.6
tasks_for_resource = [[r, t, (1 if random.random() < avg_percent_of_tasks_for_resource else 0)]for r in resources.index for t in tasks.index ]
tasks_for_resource = pd.DataFrame(data=tasks_for_resource, columns=['Resource', 'Task', 'CanPerform']).set_index(['Resource', 'Task']).query('CanPerform == 1')
tasks_for_resource

CanPerform
Resource Task            
A        1              1
         4              1
         6              1
         8              1
         10             1
...                   ...
F        13             1
         14             1
         15             1
         16             1
         20             1

[68 rows x 1 columns]

In [9]:
with pd.ExcelWriter('../data/data.xlsx') as xlsxwriter:
    tasks.to_excel(xlsxwriter, 'Tasks', merge_cells=False)
    resource_groups.to_excel(xlsxwriter, 'Resource Groups', merge_cells=False)
    resources.to_excel(xlsxwriter, 'Resources', merge_cells=False)
    tasks_for_resource.to_excel(xlsxwriter, 'Tasks For Resources', merge_cells=False)